# **IMPORTACION DE PAQUETES**
---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# **Exploración de los datos**
---

In [ ]:
data=pd.read_csv('./Data/dataset_para_modelar.csv')
data=data.iloc[:,1:]
data.shape

In [ ]:
data.head()

In [ ]:
data.info()

- Vemos que edad esta en tipo texto
- Hay muchos nulos en la columna 10

### **Comprobamos el numero de nulos**

In [ ]:
data.isna().sum()

- Vemos que hay 56 que son recurrentes, vamos a confirmarlo

In [ ]:
data[data['pais'].isna()].isna().sum()

- Efectivamente hay 56 usuarios cuya informacion es nula para casi todas las columnas.

### **Inspección de cada variable**

##### - **Código de persona**

In [ ]:
cods_persona=data[['cod_persona','mes']]
len(cods_persona['cod_persona'].unique()) #Número de personas

Tenemos un total de 46779 usuarios.

- ¿Evoluciona el numero de personas con el tiempo?

In [ ]:
user_per_mes=cods_persona.groupby(by=['mes'])['cod_persona'].count().reset_index()
user_per_mes['mes']=pd.to_datetime(user_per_mes['mes'])
sns.scatterplot(data=user_per_mes,x='mes',y='cod_persona')
plt.xticks(rotation=45)

- Curiosamente a partir de Julio de 2015 parece que se unen muchos usuarios, esto puede deberse a que la empresa haya sido absorbida por otra.

#### - **País**

In [ ]:
sns.barplot(data['pais'].value_counts())
plt.xticks(rotation=45)

- Parece que la mayoria son de españa vamos a verificarlo

In [ ]:
data['pais'].value_counts() #/ len(data)

- Vemos que el 99,5% son español, por lo tanto quizas sea interesante convertir la columna en español o no.

#### - **Sexo**

In [ ]:
sns.barplot(data['sexo'].value_counts())

- Vemos que hay practicamente la misma cantidad de hombres y mujeres.

#### - **Edad**
El primer caso es entender que errores han llevado a que la considere como texto

In [ ]:
data['edad'].unique()

Observamos varios valores extraños:
- ' NA'
- Algunos numeros tienen espacios
  
Para resolverlo vamos primero a eliminar espacios, sustituir el NA por un nulo y convertir a numérico

In [ ]:
data['edad']=data['edad'].apply(lambda x: np.nan if x==' NA'else x).astype(np.float32)
data['edad'].unique()

- Vemos que hemos solucionado el problema ahora vamos a estudiar como se distribuyen las edades y si evolucionan con el tiempo

In [ ]:
sns.histplot(data['edad'])

Podemos observar 3 grupos:
- Menores de 20 años (menores)
- Entre 20 y 30 años (jovenes)
- +30 años (Adultos)

In [ ]:
sns.boxplot(data['edad']) #Parece que hay algun valor de edad que podria ser erroneo

In [ ]:
edad_media_mes=data[['edad','mes']].groupby(by=['mes'])['edad'].median().reset_index() #Probar tambien con la mediana y explicar la diferencia
edad_media_mes['mes']=pd.to_datetime(user_per_mes['mes'])
sns.scatterplot(data=edad_media_mes,x='mes',y='edad')
plt.xticks(rotation=45)

- Parece que la media apenas varia con el tiempo

#### - **xti_empleado**

In [ ]:
data['xti_empleado'].unique() #Es una variable categorica

In [ ]:
sns.barplot(data['xti_empleado'].value_counts()) 

- Vemos que la mayoria pertenecen a la categoria N, teniendo en cuenta la descripcion de las columnas la mayoria de los clientes no son empleados lo cual es lógico:
- Descripcion variable: Employee index: A active, B ex employed, F filial, N not employee, P pasive

Quizas sería interesante convertir en empleado/no empleado

#### - **xti_nuevo_cliente**

In [ ]:
data['xti_nuevo_cliente'].unique() 

Es una variable dicotómica si/no.
- Descripcion de la variable: New customer Index. 1 if the customer registered in the last 6 months.

¿Que cantidad hay de nuevos?

In [ ]:
sns.barplot(data['xti_nuevo_cliente'].value_counts())

- Vemos que la mayoria no son clientes nuevos

#### - **num_antiguedad**

In [ ]:
data['num_antiguedad'].unique()

La antiguedad esta en meses
Problemas que observamos:
- Uso de espacios
- '     NA'

In [ ]:
data['num_antiguedad']=data['num_antiguedad'].apply(lambda x: np.nan if x=='     NA' else x).astype(np.float32)
data['num_antiguedad'].unique()

In [ ]:
data['num_antiguedad'].describe()

In [ ]:
sns.histplot(data['num_antiguedad'])

In [ ]:
sns.boxplot(data['num_antiguedad'])

#### - **xti_rel**

In [ ]:
data['xti_rel'].unique()

Parece una variable categorica donde las categorias se han escrito como numeros.
Para que las categorias tengan una escala similar, sustituimos 99 por 2.

In [ ]:
data['xti_rel']=data['xti_rel'].map({99.:2.,1.:1.})

In [ ]:
sns.barplot(data['xti_rel'].value_counts())

Descripcion de la variable: 1 (First/Primary), 99 (Primary customer during the month but not at the end of the month)
- Vemos que la mayoria pertenecen a la categoria 1

#### - **fec_ult_cli_1t**

In [ ]:
data['fec_ult_cli_1t'].unique()

In [ ]:
data['fec_ult_cli_1t'].isna().sum()

- Parece que se trata de una columna de fechas pero la mayoria son nulos por lo tanto convendria eliminarla

#### - **xti_rel_1mes**

In [ ]:
data['xti_rel_1mes'].unique()

Se trata de una variable categorica donde las categorias son numeros:
- Descripcion de la variable: Customer type at the beginning of the month ,1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner)

In [ ]:
sns.barplot(data['xti_rel_1mes'].value_counts())

- La gran mayoria pertenecen a la categoria 1

#### - **tip_rel_1mes**

In [ ]:
data['tip_rel_1mes'].unique()

In [ ]:
sns.barplot(data['tip_rel_1mes'].value_counts())

- Descripcion de la variable: Customer relation type at the beginning of the month, A (active), I (inactive), P (former customer),R (Potential)

#### - **indresi**

In [ ]:
data['indresi'].unique()

In [ ]:
sns.barplot(data['indresi'].value_counts())

La gran mayoria pertenece a la categoria S, no quiere decir que la mayoria de usuarios tienen la cuenta del banco en el mismo pais que donde viven:
- Descripcion de la variable: esidence index (S (Yes) or N (No) if the residence country is the same than the bank country)

#### - **indext**

In [ ]:
data['indext'].unique()

In [ ]:
sns.barplot(data['indext'].value_counts())

- Descripcion de la variable: Foreigner index (S (Yes) or N (No) if the customer's birth country is different than the bank country)

#### - **des_canal**

In [ ]:
data['des_canal'].unique()

- Descripcion de la variable: channel used by the customer to join

In [ ]:
sns.barplot(data['des_canal'].value_counts())

In [ ]:
data['des_canal'].value_counts()

- Parece que los datos se concentran en las primeras categorias

#### - **xti_extra**

In [ ]:
data['xti_extra'].unique()

In [ ]:
sns.barplot(data['xti_extra'].value_counts())

La gran mayoria pertenece a la categoria 'N':
- Descripcion de la variable: Deceased index. N/S

No parece que nos de mucha informacion

#### - **tip_dom**

In [ ]:
data['tip_dom'].unique()

In [ ]:
sns.barplot(data['tip_dom'].value_counts())

- Solo hay valores de una categoria lo cual no va a añadir informacion a los posteriores modelos ya que la varianza en la informacion es 0.
  
#### - **cod_provincia**

In [ ]:
data['cod_provincia'].unique()

- Descripcion de la variable: Province code (customer's address)

#### - **xti_actividad_cliente**

In [ ]:
data['xti_actividad_cliente'].unique()

In [ ]:
sns.barplot(data['xti_actividad_cliente'].value_counts())

- Descripcion de las variables: Activity index (1, active customer; 0, inactive customer)
- Observamos que hay practicamente el mismo numero de clientes activoc como de inactivos

#### - **imp_renta**

In [ ]:
data['imp_renta'].unique()

In [ ]:
sns.histplot(data['imp_renta'])

- Descripcion de la variable: Gross income of the household

Vemos que hay valores atípicos que nos modifican la distribucion, probemos a normalizarla con logaritmos

In [ ]:
sns.histplot(np.log(data['imp_renta']),kde=True)

Observamos que ahora la distribucion si esta normalizada, quizas seria interesante aplicar un logaritmo tras tratar los valores nulos

In [ ]:
sns.boxplot(data['imp_renta']) #Así vemos mas claramente los valores atipicos

In [ ]:
data['imp_renta'].describe()

#### - **id_segmento**

In [ ]:
data['id_segmento'].unique()

- Descripcion de la variable:  	segmentation: 01 - VIP, 02 - Individuals 03 - college graduated

In [ ]:
sns.barplot(data['id_segmento'].value_counts())

#### - **mean_engagement**

In [ ]:
sns.histplot(data['mean_engagement'])

In [ ]:
sns.boxplot(data['mean_engagement'])

- Observamos que hay muchos valores atípicos respecto a la media

In [ ]:
data['mean_engagement'].describe()

## **Estudio del comportamiento de compra y los productos**
---

El primer paso es corregir los valores faltantes de los productos

In [ ]:
inds_prods=[f'ind_prod{i}' for i in range(1,26)]
informacion_productos=data.sort_values(by=['cod_persona','mes'])[inds_prods]

In [ ]:
informacion_productos.isna().sum()

Vamos a rellenar los valores nulos con el valor del periodo anterior, suponemos que mantiene el mismo estado que para el mes anterio.

In [ ]:
informacion_productos.fillna(method='ffill',inplace=True)
informacion_productos.isna().sum()

Vamos a comenzar calculando las reglas de asociacion entre los productos, estas nos vienen a decir que productos se suelen comprar juntos lo cual nos puede dar informacion para mejorar las recomendaciones.

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(informacion_productos, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Display the association rules
rules.sort_values(by='lift',ascending=False)

Vemos que para aquellos que han comprado los producto 24,22 y 25 generalmente compran tambien el 23,19 y 5.

### **Estudio respecto al grupo de edad**

En principio suponemos que es probable que en funcion de su edad podra acceder a unos productos como una hipoteca o a otros

In [ ]:
data['edad'].fillna(data['edad'].median(),inplace=True) #Rellenamos los nulos con la mediana

In [ ]:
data['edad_dividida'] = pd.cut(data['edad'].astype(np.int32), bins=[-np.inf, 20, 30, np.inf], labels=['Menor', 'Joven','Adulto'])
data.groupby('edad_dividida')[inds_prods].sum()

Resumen conclusiones:
- Los menores no compran los productos 1,2,4,8,9,...,16,17,21
- Los Jovenes no compran los productos 1,2,4,6 y 21.
- Los Adultos no compran los productos 2 y 6.

Vemos que el producto 2 no se compra nunca por lo tanto lo lógico sería no recomendarlo.

In [ ]:
data['ind_prod2'].describe() #Vemos que son solo 0 siempre (nadie lo ha comprado)

#### **Estudio respecto al grupo de segmento**

In [ ]:
data['id_segmento'].isna().sum()

Vamos a rellenar los valores nulos con el segmento mas comun.

In [ ]:
from sklearn.impute import SimpleImputer
seg_imputer=SimpleImputer(strategy='most_frequent')
data['id_segmento']=seg_imputer.fit_transform(data[['id_segmento']])

In [ ]:
dict(data.groupby('id_segmento')[inds_prods].sum())

Resumen de conclusiones:
- Los usuarios del segmento TOP no compran: producto 1,2,6.
- Los usuarios del segmento PARTICULARES no compran: 2
- los usuarios del segmento UNIVERSITARIO no compran: 1,2,6,15 y 21.

En conclusion habria que comprobar los resultados del modelo impleando ambas aproximaciones, agrupando por edad y agrupando por segmento.

# **ACABAR MAÑANA VIENDO LA EVOLUCION DE LOS DIFERENTES PRODUCTOS**

## **Estudio de los clientes (Clustering)**
---

Para este analisis vamos a realizarnos una copia de los datos (eliminando los productos) y trataremos de forma preliminar los nulos.

In [ ]:
data_cluster=data.drop(inds_prods,axis=1).copy()
data_cluster.head()

Para convertir las categorias a datos numericos, como no siguen un orden implicito (bueno,regular,malo) en lugar de emplear codificacion ordenada haremos label enconder.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
num_cols=data_cluster.select_dtypes(include='number').columns
cat_cols=data_cluster.select_dtypes(exclude='number').columns

Rellenamos los nulos categoricos con la categoria mas frecuente

In [ ]:
cat_imputer=SimpleImputer(strategy='most_frequent')
data_cluster[cat_cols]=cat_imputer.fit_transform(data_cluster[cat_cols])

Rellenamos los nulos numericos con la mediana

In [ ]:
num_imputer=SimpleImputer(strategy='median')
data_cluster[num_cols]=num_imputer.fit_transform(data_cluster[num_cols])

Eliminamos las columnas que habiamos visto en el analisis que no aportaban informacion:
- fec_ult_cli_1t muchos valores nulos
- xti_extra no tiene varianza, da poca informacion

Despues de este analisis comprobaremos que variables son mas y menos importantes

In [ ]:
data_cluster.drop(['fec_ult_cli_1t','xti_extra'],axis=1,inplace=True)

Ahora que hemos corregido los nulos y hemos eliminado las columnas innecesarias, Codificamos las variables categoricas con labelEncoder.

In [ ]:
cat_cols_en=data_cluster.select_dtypes(exclude='number').columns
label_encoder=OrdinalEncoder()
data_cluster[cat_cols_en]=label_encoder.fit_transform(data_cluster[cat_cols_en])

In [ ]:
data_cluster.head()

Vemos que ya hemos convertido todas las variables a numericas, vamos a eliminar las columnas que no dan informacion sobre el comportamiento de grupos de cliente:
- cod_persona
- mes
- fecha1

In [ ]:
data_cluster.drop(['cod_persona','mes','fecha1'],axis=1,inplace=True)

#### - **El paso inicial es encontrar el numero óptimo de grupos**
Empezamos con la técnica del codo.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,calinski_harabasz_score

In [ ]:
# definimos la n-tupla donde ordenaderemos los sucesivos valores de SSE
# hallados para cada valor de k
Coste = []
krange = np.arange(2, 11) #krange = 2,3,4,5,6,7,8,9,10
# bucle para el cálculo de la función de coste (SSE) desde k= 2 hasta k= 10
for num in krange:
    kmeans = KMeans(n_clusters=num, n_init='auto', random_state=10, max_iter=100).fit(data_cluster)
    print(
        "Para k =",
        num,
        ', el coste (SSE)=',
        kmeans.inertia_,
    )
    Coste.append(kmeans.inertia_)

fig, ax = plt.subplots()
ax.set_title('Método del codo', fontsize = 16)
ax.set_ylabel("Coste (SSE)",fontsize=14)
ax.set_xlabel("Número de clústeres (k)", fontsize=14)
ax.plot(krange,Coste)
plt.show()

Tratamos de encontrar el numeo de cluster que forma el codo que podria estar entre 3-6, vamos a emplear una nueva métrica para calcularlo matematicamente.

In [ ]:
for k in [3,4,5,6]:
    kmeans = KMeans(n_clusters=k, n_init= 'auto', random_state=10, max_iter=3000)
    Y_pred=kmeans.fit_predict(data_cluster) # Vector de asignación de etiquetas predichas para cada elemento
    data_cluster['id_cluster']=kmeans.labels_
    muestra=data_cluster.sample(round(0.1*len(data_cluster)))
    silhouette_avg = silhouette_score(muestra.drop('id_cluster',axis=1),muestra['id_cluster'])
    cal=calinski_harabasz_score(muestra.drop('id_cluster',axis=1),muestra['id_cluster'])
    print('\nPara un Nº de clusters: ',k ,
          '\nS: ',silhouette_avg,
          '\nCH: ',cal)
    print('-'*50)

Tratamos de buscar el valor mas cercano a 1 de la metrica de silouhette (S) y maximizar calinski (CH), teniendo esto en cuenta el numero optimo son 3 clusters.
Vamos a entrenar el modelo con 3 cluster y analizar los grupos.

In [ ]:
k_val=3
kmeans = KMeans(n_clusters=k_val, n_init= 'auto', random_state=10, max_iter=3000)
kmeans.fit(data_cluster) # Vector de asignación de etiquetas predichas para cada elemento
data_cluster['id_cluster']=kmeans.labels_

Vamos a mostrar con diagramas de cajas cada grupo para todas las variables.

In [ ]:
for col in data_cluster.columns:
    sns.boxplot(data_cluster,x=col,hue='id_cluster')
    plt.show()

In [ ]:
data_cluster['id_cluster'].value_counts()

Vemos que apenas aporta informacion ya que no parece haber agrupaciones muy claras.

# **Limpieza de datos y prueba con varios modelos**
---

Con toda la informacion obtenida durante el analisis, vamos a limpiar los datos y evaluar diferentes modelos a ver cual funciona mejor.

- Convertimos pais a español o no y rellenamos nulos con el mas frecuente.

In [ ]:
data['pais']=data['pais'].fillna('ES').apply(lambda x: 0 if x=='ES' else 1 )

- Convertimos sexo a numerico y rellenamos con el mas frecuente (masculino)

In [ ]:
mapeo_sexo={'V':0,'H':1}
data['sexo']=data['sexo'].fillna('V').map(mapeo_sexo)

- xti_empleado rellenamos con la mas frecuente y convertimos a numerico

In [ ]:
map_xti_emp={'N':0,'A':1,'F':2,'B':3}
data['xti_empleado']=data['xti_empleado'].fillna('N').map(map_xti_emp)

- Rellenamos los nulos con la categoria mas frecuente.

In [ ]:
data['xti_nuevo_cliente'].fillna(0,inplace=True)

- Rellenamos el numero de antiguedad con la media.

In [ ]:
data['num_antiguedad'].fillna(data['num_antiguedad'].median(),inplace=True)

- Rellenamos con la categoria mas frecuente (1)

In [ ]:
data['xti_rel'].fillna(1.,inplace=True)

- Eliminamos la columna porque teniamos muchos valores nulos +99%

In [ ]:
data.drop('fec_ult_cli_1t',axis=1,inplace=True)

- Rellenamos con la categoria más común (1)

In [ ]:
data['xti_rel_1mes'].fillna(1.,inplace=True)

- Rellenamos con la categoria mas frecuente 'I' y mapeamos a numerico

In [ ]:
map_tip_rel={'I':0,'A':1,'P':2,'R':3}
data['tip_rel_1mes']=data['tip_rel_1mes'].fillna('I').map(map_tip_rel)

- Rellenamos con la categoria mas comun 'S' y mapeamos a numerico

In [ ]:
map_indresi={'S':0,'N':1}
data['indresi']=data['indresi'].fillna('S').map(map_indresi)

In [ ]:
map_indext={'S':0,'N':1}
data['indext']=data['indext'].fillna('N').map(map_indext)

- Rellenamos los nulos con la mas comun (KHE) y convertimos a numerico 

In [ ]:
map_canal={elem:id for id,elem in enumerate(data['des_canal'].unique())}
data['des_canal']=data['des_canal'].fillna('KHE').map(map_canal)

- Rellenamos con la mas común y convertimos a numerico

In [ ]:
map_xti_extra={'S':0,'N':1}
data['xti_extra']=data['xti_extra'].fillna('N').map(map_xti_extra)

- Todos pertenecen a la misma categoria por lo tanto no nos da informacion, la podemos eliminar.

In [ ]:
data.drop('tip_dom',axis=1,inplace=True)

- Rellenamos con el mas comun porque aunque sea numerica el codigo postal en una categoria.

In [ ]:
data['cod_provincia'].fillna(28.,inplace=True)

- Rellenamos con la mas comun (0)

In [ ]:
data['xti_actividad_cliente'].fillna(0.,inplace=True)

- Rellenamos con la mediana y hacemos una transformacion logaritmica para estandarizar la distribucion.

In [ ]:
data['imp_renta']=np.log(data['imp_renta'].fillna(data['imp_renta'].median()))

- Rellenamos con la categoria mas común y convertimos a numerico

In [ ]:
map_seg={elem:id for id,elem in enumerate(data['id_segmento'].unique())}
data['id_segmento']=data['id_segmento'].fillna('02 - PARTICULARES').map(map_seg)

- Rellenamos con la mediana

In [ ]:
data['mean_engagement'].fillna(data['mean_engagement'].median(),inplace=True)

In [ ]:
map_edad_div={elem:id for id,elem in enumerate(data['edad_dividida'].unique())}
data['edad_dividida']=data['edad_dividida'].fillna('Adulto').map(map_edad_div)

- Para la fecha uno hay que rellenar con el valor anterior + 1 mes

In [ ]:
data['fecha1'] = pd.to_datetime(data['fecha1'])

# Función para agregar un mes a una fecha
def add_month(date):
    if date.month == 12:
        return date.replace(year=date.year + 1, month=1)
    else:
        return date.replace(month=date.month + 1)

# Rellenar los valores nulos
for i in range(1, len(data)):
    if pd.isna(data.loc[i, 'fecha1']):
        data.loc[i, 'fecha1'] = add_month(data.loc[i - 1, 'fecha1'])

data['fecha1'].isna().sum()

Revisamos que no haya ningun nulo que se nos haya olvidado.

In [ ]:
data.isna().sum()

Vemos que se nos habia olvidado rellenar los valores de los producto procedemos igual que antes.

In [ ]:
data['ind_prod22'].fillna(method='ffill',inplace=True)
data['ind_prod23'].fillna(method='ffill',inplace=True)

 #### Ahora si! Ya tenemos la limpieza inicial, nos guardamos nuestro set de datos limpio y vamos con los modelos!!

In [ ]:
data.to_csv('datos_limpios.csv',index=False)